In [1]:
import torch
from fastchat.conversation import Conversation, SeparatorStyle
import pandas as pd

In [2]:
from alpaca_lora_4bit.autograd_4bit import load_llama_model_4bit_low_ram, Autograd4bitQuantLinear, switch_backend_to
switch_backend_to("cuda")

Triton not found. Please run "pip install triton".


c:\Users\alex4321\anaconda3\envs\llama\lib\site-packages\tqdm-4.65.0-py3.10.egg\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Using CUDA implementation.


In [3]:
import alpaca_lora_4bit.matmul_utils_4bit
alpaca_lora_4bit.matmul_utils_4bit.act_order = False

In [4]:
model, tokenizer = load_llama_model_4bit_low_ram("vicuna-7B-GPTQ-4bit-128g",
                                                 "vicuna-7B-GPTQ-4bit-128g/vicuna-7B-GPTQ-4bit-128g.safetensors",
                                                 groupsize=128)
model.half()
for n, m in model.named_modules():
    if isinstance(m, Autograd4bitQuantLinear):
        if m.is_v1_model:
            m.zeros = m.zeros.half()
        m.scales = m.scales.half()
        m.bias = m.bias.half()
from alpaca_lora_4bit.amp_wrapper import AMPWrapper
wrapper = AMPWrapper(model)
wrapper.apply_generate()

Loading Model ...


The model weights are not tied. Please use the `tie_weights` method before using the `infer_auto_device` function.
The model weights are not tied. Please use the `tie_weights` method before using the `infer_auto_device` function.
The safetensors archive passed at vicuna-7B-GPTQ-4bit-128g/vicuna-7B-GPTQ-4bit-128g.safetensors does not contain metadata. Make sure to save your model with the `save_pretrained` method. Defaulting to 'pt' metadata.


Loaded the model in 4.65 seconds.


In [6]:
for i in range(10):
    torch.manual_seed(42 + i)
    x = torch.rand((1, model.config.hidden_size)).half().to(model.device)
    alpaca_lora_4bit.matmul_utils_4bit.faster_mode = "disabled"
    with torch.no_grad():
        output_disabled = model.model.layers[0].self_attn.q_proj.forward(x)
    alpaca_lora_4bit.matmul_utils_4bit.faster_mode = "old_faster"
    with torch.no_grad():
        output_old_faster = model.model.layers[0].self_attn.q_proj.forward(x)
    alpaca_lora_4bit.matmul_utils_4bit.faster_mode = "faster"
    with torch.no_grad():
        output_faster = model.model.layers[0].self_attn.q_proj.forward(x)
    mae_to_faster = torch.abs(output_disabled - output_faster).mean()
    mae_to_old_faster = torch.abs(output_disabled - output_old_faster).mean()
    mae_faster_old_faster = torch.abs(output_old_faster - output_faster).mean()
    print("DISABLED-FASTER", mae_to_faster.item(), "\nDISABLED-OLD FASTER", mae_to_old_faster.item(), "\nFASTER-OLD FASTER", mae_faster_old_faster.item())
    output_disabled_series = pd.Series(output_disabled.view(-1).detach().cpu().numpy())
    print("DISABLED OUTPUT (5% - 95% quantiles)", output_disabled_series.quantile(0.05), output_disabled_series.quantile(0.95))
    print("\n\n")

DISABLED-FASTER 1.0654296875 
DISABLED-OLD FASTER 0.86083984375 
FASTER-OLD FASTER 0.90478515625
DISABLED OUTPUT (5% - 95% quantiles) -2.06591796875 2.02783203125



DISABLED-FASTER 1.0927734375 
DISABLED-OLD FASTER 0.93994140625 
FASTER-OLD FASTER 0.86962890625
DISABLED OUTPUT (5% - 95% quantiles) -2.06787109375 2.0029296875



DISABLED-FASTER 1.20703125 
DISABLED-OLD FASTER 0.9873046875 
FASTER-OLD FASTER 0.99951171875
DISABLED OUTPUT (5% - 95% quantiles) -1.97216796875 2.03076171875



DISABLED-FASTER 1.0576171875 
DISABLED-OLD FASTER 0.85595703125 
FASTER-OLD FASTER 0.86328125
DISABLED OUTPUT (5% - 95% quantiles) -1.88232421875 1.8505859375



DISABLED-FASTER 1.115234375 
DISABLED-OLD FASTER 0.98388671875 
FASTER-OLD FASTER 0.97265625
DISABLED OUTPUT (5% - 95% quantiles) -1.98876953125 1.958251953125



DISABLED-FASTER 1.1455078125 
DISABLED-OLD FASTER 0.87109375 
FASTER-OLD FASTER 0.92919921875
DISABLED OUTPUT (5% - 95% quantiles) -2.00439453125 2.01318359375



DISABLED-FASTER 1.